In [102]:
from sqlalchemy import create_engine

engine = create_engine('sqlite:///:memory:', echo=True)
engine

Engine(sqlite:///:memory:)

In [103]:
from sqlalchemy.orm import DeclarativeBase

class Base(DeclarativeBase):
    pass

### Mapped classes - Animal

In [104]:
from sqlalchemy.orm import Mapped, mapped_column, relationship
from sqlalchemy import String, ForeignKey
from typing import Optional, List

class Pessoa(Base):
    __tablename__ = 'PESSOA'

    id: Mapped[int] = mapped_column(primary_key=True)
    nome: Mapped[str] = mapped_column(String(50))
    sobrenome: Mapped[Optional[str]]

    animais: Mapped[List['Animal']] = relationship(back_populates='pessoa')

    def __repr__(self) -> str:
        return f'Pessoa(id={self.id!r}, nome={self.nome!r}, sobrenome={self.sobrenome!r}, animais={self.animais!r})'


class Animal(Base):
    __tablename__ = 'ANIMAL'

    id: Mapped[int] = mapped_column(primary_key=True)
    nome: Mapped[str] = mapped_column(String(50))
    id_pessoa = mapped_column(ForeignKey('PESSOA.id'))

    pessoa: Mapped[Pessoa] = relationship(back_populates='animais')

    def __repr__(self) -> str:
        return f'Animal(id={self.id!r}, nome={self.nome!r}, id_pessoa={self.id_pessoa!r}, pessoa={self.pessoa.nome!r})'

In [105]:
Base.metadata.create_all(engine)

2023-11-30 20:35:14,616 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-30 20:35:14,617 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("PESSOA")
2023-11-30 20:35:14,617 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-11-30 20:35:14,618 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("PESSOA")
2023-11-30 20:35:14,618 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-11-30 20:35:14,619 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("ANIMAL")
2023-11-30 20:35:14,619 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-11-30 20:35:14,619 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("ANIMAL")
2023-11-30 20:35:14,620 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-11-30 20:35:14,621 INFO sqlalchemy.engine.Engine 
CREATE TABLE "PESSOA" (
	id INTEGER NOT NULL, 
	nome VARCHAR(50) NOT NULL, 
	sobrenome VARCHAR, 
	PRIMARY KEY (id)
)


2023-11-30 20:35:14,622 INFO sqlalchemy.engine.Engine [no key 0.00068s] ()
2023-11-30 20:35:14,622 INFO sqlalchemy.engine.Engine 
CREATE TA

In [106]:
from sqlalchemy import inspect

inspect(engine).get_table_names()

2023-11-30 20:35:14,629 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-30 20:35:14,630 INFO sqlalchemy.engine.Engine SELECT name FROM sqlite_master WHERE type='table' AND name NOT LIKE 'sqlite~_%' ESCAPE '~' ORDER BY name
2023-11-30 20:35:14,630 INFO sqlalchemy.engine.Engine [raw sql] ()
2023-11-30 20:35:14,632 INFO sqlalchemy.engine.Engine ROLLBACK


['ANIMAL', 'PESSOA']

In [107]:
jubileu = Pessoa(nome='Jubileu')
jubileu

Pessoa(id=None, nome='Jubileu', sobrenome=None, animais=[])

In [108]:
jose_pocai = Pessoa(nome='José', sobrenome='Pocai')
jose_pocai

Pessoa(id=None, nome='José', sobrenome='Pocai', animais=[])

In [109]:
from sqlalchemy.orm import Session

session = Session(engine, expire_on_commit=False)
session

In [110]:
session.add_all([jubileu, jose_pocai])

In [111]:
session.new #ver o que está no stage

IdentitySet([Pessoa(id=None, nome='Jubileu', sobrenome=None, animais=[]), Pessoa(id=None, nome='José', sobrenome='Pocai', animais=[])])

In [112]:
session.commit()

2023-11-30 20:35:14,677 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-30 20:35:14,679 INFO sqlalchemy.engine.Engine INSERT INTO "PESSOA" (nome, sobrenome) VALUES (?, ?) RETURNING id
2023-11-30 20:35:14,679 INFO sqlalchemy.engine.Engine [generated in 0.00010s (insertmanyvalues) 1/2 (ordered; batch not supported)] ('Jubileu', None)
2023-11-30 20:35:14,680 INFO sqlalchemy.engine.Engine INSERT INTO "PESSOA" (nome, sobrenome) VALUES (?, ?) RETURNING id
2023-11-30 20:35:14,680 INFO sqlalchemy.engine.Engine [insertmanyvalues 2/2 (ordered; batch not supported)] ('José', 'Pocai')
2023-11-30 20:35:14,681 INFO sqlalchemy.engine.Engine COMMIT


In [113]:
print(jose_pocai)
jubileu

2023-11-30 20:35:14,688 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-30 20:35:14,689 INFO sqlalchemy.engine.Engine SELECT "ANIMAL".id AS "ANIMAL_id", "ANIMAL".nome AS "ANIMAL_nome", "ANIMAL".id_pessoa AS "ANIMAL_id_pessoa" 
FROM "ANIMAL" 
WHERE ? = "ANIMAL".id_pessoa
2023-11-30 20:35:14,690 INFO sqlalchemy.engine.Engine [generated in 0.00065s] (2,)
Pessoa(id=2, nome='José', sobrenome='Pocai', animais=[])
2023-11-30 20:35:14,691 INFO sqlalchemy.engine.Engine SELECT "ANIMAL".id AS "ANIMAL_id", "ANIMAL".nome AS "ANIMAL_nome", "ANIMAL".id_pessoa AS "ANIMAL_id_pessoa" 
FROM "ANIMAL" 
WHERE ? = "ANIMAL".id_pessoa
2023-11-30 20:35:14,692 INFO sqlalchemy.engine.Engine [cached since 0.002863s ago] (1,)


Pessoa(id=1, nome='Jubileu', sobrenome=None, animais=[])

In [114]:
session.close()

2023-11-30 20:35:14,698 INFO sqlalchemy.engine.Engine ROLLBACK


In [115]:
with Session(engine) as session:
    print(session.get(Pessoa, 1))

2023-11-30 20:35:14,705 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-30 20:35:14,706 INFO sqlalchemy.engine.Engine SELECT "PESSOA".id AS "PESSOA_id", "PESSOA".nome AS "PESSOA_nome", "PESSOA".sobrenome AS "PESSOA_sobrenome" 
FROM "PESSOA" 
WHERE "PESSOA".id = ?
2023-11-30 20:35:14,707 INFO sqlalchemy.engine.Engine [generated in 0.00084s] (1,)
2023-11-30 20:35:14,708 INFO sqlalchemy.engine.Engine SELECT "ANIMAL".id AS "ANIMAL_id", "ANIMAL".nome AS "ANIMAL_nome", "ANIMAL".id_pessoa AS "ANIMAL_id_pessoa" 
FROM "ANIMAL" 
WHERE ? = "ANIMAL".id_pessoa
2023-11-30 20:35:14,709 INFO sqlalchemy.engine.Engine [cached since 0.01992s ago] (1,)
Pessoa(id=1, nome='Jubileu', sobrenome=None, animais=[])
2023-11-30 20:35:14,710 INFO sqlalchemy.engine.Engine ROLLBACK


In [116]:
from sqlalchemy import select

print(select(Pessoa))

SELECT "PESSOA".id, "PESSOA".nome, "PESSOA".sobrenome 
FROM "PESSOA"


In [117]:
with Session(engine) as session:
    result = session.scalars(select(Pessoa))

    for pessoa in result:
        print(f'{pessoa.id} -> {pessoa.nome} {pessoa.sobrenome}')

2023-11-30 20:35:14,722 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-30 20:35:14,724 INFO sqlalchemy.engine.Engine SELECT "PESSOA".id, "PESSOA".nome, "PESSOA".sobrenome 
FROM "PESSOA"
2023-11-30 20:35:14,725 INFO sqlalchemy.engine.Engine [generated in 0.00087s] ()
1 -> Jubileu None
2 -> José Pocai
2023-11-30 20:35:14,726 INFO sqlalchemy.engine.Engine ROLLBACK


In [118]:
with Session(engine) as session:
    cintia = Pessoa(nome='Cintia', sobrenome='Carvalho')
    session.add(cintia)

    print(cintia in session)

True


In [119]:
with Session(engine) as session:
    print(session.execute(select(Pessoa).filter_by(nome='José')).scalar_one())
    print(session.scalar(select(Pessoa).filter_by(nome='Jubileu')))

2023-11-30 20:35:14,748 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-30 20:35:14,749 INFO sqlalchemy.engine.Engine SELECT "PESSOA".id, "PESSOA".nome, "PESSOA".sobrenome 
FROM "PESSOA" 
WHERE "PESSOA".nome = ?
2023-11-30 20:35:14,750 INFO sqlalchemy.engine.Engine [generated in 0.00079s] ('José',)
2023-11-30 20:35:14,751 INFO sqlalchemy.engine.Engine SELECT "ANIMAL".id AS "ANIMAL_id", "ANIMAL".nome AS "ANIMAL_nome", "ANIMAL".id_pessoa AS "ANIMAL_id_pessoa" 
FROM "ANIMAL" 
WHERE ? = "ANIMAL".id_pessoa
2023-11-30 20:35:14,752 INFO sqlalchemy.engine.Engine [cached since 0.06313s ago] (2,)
Pessoa(id=2, nome='José', sobrenome='Pocai', animais=[])
2023-11-30 20:35:14,753 INFO sqlalchemy.engine.Engine SELECT "PESSOA".id, "PESSOA".nome, "PESSOA".sobrenome 
FROM "PESSOA" 
WHERE "PESSOA".nome = ?
2023-11-30 20:35:14,754 INFO sqlalchemy.engine.Engine [cached since 0.004398s ago] ('Jubileu',)
2023-11-30 20:35:14,755 INFO sqlalchemy.engine.Engine SELECT "ANIMAL".id AS "ANIMAL_id", "ANIMAL".

In [120]:
session = Session(engine)

jubileu = session.get(Pessoa, 1)
jubileu

2023-11-30 20:35:14,762 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-30 20:35:14,763 INFO sqlalchemy.engine.Engine SELECT "PESSOA".id AS "PESSOA_id", "PESSOA".nome AS "PESSOA_nome", "PESSOA".sobrenome AS "PESSOA_sobrenome" 
FROM "PESSOA" 
WHERE "PESSOA".id = ?
2023-11-30 20:35:14,764 INFO sqlalchemy.engine.Engine [cached since 0.05711s ago] (1,)
2023-11-30 20:35:14,765 INFO sqlalchemy.engine.Engine SELECT "ANIMAL".id AS "ANIMAL_id", "ANIMAL".nome AS "ANIMAL_nome", "ANIMAL".id_pessoa AS "ANIMAL_id_pessoa" 
FROM "ANIMAL" 
WHERE ? = "ANIMAL".id_pessoa
2023-11-30 20:35:14,766 INFO sqlalchemy.engine.Engine [cached since 0.07666s ago] (1,)


Pessoa(id=1, nome='Jubileu', sobrenome=None, animais=[])

In [121]:
jubileu.nome = 'Lorival'

In [122]:
session.dirty

IdentitySet([Pessoa(id=1, nome='Lorival', sobrenome=None, animais=[])])

In [123]:
jubileu_sobrenome = session.scalar(select(Pessoa.sobrenome).where(Pessoa.id == 1))
jubileu_sobrenome

2023-11-30 20:35:14,786 INFO sqlalchemy.engine.Engine UPDATE "PESSOA" SET nome=? WHERE "PESSOA".id = ?
2023-11-30 20:35:14,787 INFO sqlalchemy.engine.Engine [generated in 0.00080s] ('Lorival', 1)
2023-11-30 20:35:14,788 INFO sqlalchemy.engine.Engine SELECT "PESSOA".sobrenome 
FROM "PESSOA" 
WHERE "PESSOA".id = ?
2023-11-30 20:35:14,789 INFO sqlalchemy.engine.Engine [generated in 0.00071s] (1,)


In [124]:
session.delete(jubileu)

In [125]:
session.execute(select(Pessoa).filter_by(nome = 'Lorival')).first()

2023-11-30 20:35:14,801 INFO sqlalchemy.engine.Engine DELETE FROM "PESSOA" WHERE "PESSOA".id = ?
2023-11-30 20:35:14,802 INFO sqlalchemy.engine.Engine [generated in 0.00077s] (1,)
2023-11-30 20:35:14,803 INFO sqlalchemy.engine.Engine SELECT "PESSOA".id, "PESSOA".nome, "PESSOA".sobrenome 
FROM "PESSOA" 
WHERE "PESSOA".nome = ?
2023-11-30 20:35:14,804 INFO sqlalchemy.engine.Engine [cached since 0.05519s ago] ('Lorival',)


In [126]:
cintia = Pessoa(nome='Cintia', sobrenome='Carvalho')

session.add(cintia)
session.commit()


2023-11-30 20:35:14,811 INFO sqlalchemy.engine.Engine INSERT INTO "PESSOA" (nome, sobrenome) VALUES (?, ?)
2023-11-30 20:35:14,811 INFO sqlalchemy.engine.Engine [generated in 0.00069s] ('Cintia', 'Carvalho')
2023-11-30 20:35:14,812 INFO sqlalchemy.engine.Engine COMMIT


In [127]:
cintia

2023-11-30 20:35:14,819 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-30 20:35:14,820 INFO sqlalchemy.engine.Engine SELECT "PESSOA".id AS "PESSOA_id", "PESSOA".nome AS "PESSOA_nome", "PESSOA".sobrenome AS "PESSOA_sobrenome" 
FROM "PESSOA" 
WHERE "PESSOA".id = ?
2023-11-30 20:35:14,820 INFO sqlalchemy.engine.Engine [generated in 0.00053s] (3,)
2023-11-30 20:35:14,821 INFO sqlalchemy.engine.Engine SELECT "ANIMAL".id AS "ANIMAL_id", "ANIMAL".nome AS "ANIMAL_nome", "ANIMAL".id_pessoa AS "ANIMAL_id_pessoa" 
FROM "ANIMAL" 
WHERE ? = "ANIMAL".id_pessoa
2023-11-30 20:35:14,822 INFO sqlalchemy.engine.Engine [cached since 0.1329s ago] (3,)


Pessoa(id=3, nome='Cintia', sobrenome='Carvalho', animais=[])

In [128]:
cintia.__dict__

{'_sa_instance_state': <sqlalchemy.orm.state.InstanceState at 0x28731449af0>,
 'sobrenome': 'Carvalho',
 'nome': 'Cintia',
 'id': 3,
 'animais': []}

In [129]:
session.delete(cintia)

In [130]:
session.rollback()

2023-11-30 20:35:14,838 INFO sqlalchemy.engine.Engine ROLLBACK


In [131]:
cintia

2023-11-30 20:35:14,846 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-30 20:35:14,847 INFO sqlalchemy.engine.Engine SELECT "PESSOA".id AS "PESSOA_id", "PESSOA".nome AS "PESSOA_nome", "PESSOA".sobrenome AS "PESSOA_sobrenome" 
FROM "PESSOA" 
WHERE "PESSOA".id = ?
2023-11-30 20:35:14,848 INFO sqlalchemy.engine.Engine [cached since 0.0289s ago] (3,)
2023-11-30 20:35:14,849 INFO sqlalchemy.engine.Engine SELECT "ANIMAL".id AS "ANIMAL_id", "ANIMAL".nome AS "ANIMAL_nome", "ANIMAL".id_pessoa AS "ANIMAL_id_pessoa" 
FROM "ANIMAL" 
WHERE ? = "ANIMAL".id_pessoa
2023-11-30 20:35:14,849 INFO sqlalchemy.engine.Engine [cached since 0.1613s ago] (3,)


Pessoa(id=3, nome='Cintia', sobrenome='Carvalho', animais=[])

In [132]:
cintia in session

True

In [133]:
session.close()

2023-11-30 20:35:14,863 INFO sqlalchemy.engine.Engine ROLLBACK


In [134]:
with Session(engine) as session:
    result = session.scalars(select(Pessoa))

    for pessoa in result:
        print(f'{pessoa.id} -> {pessoa.nome} -> {pessoa.sobrenome}')

2023-11-30 20:35:14,869 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-30 20:35:14,870 INFO sqlalchemy.engine.Engine SELECT "PESSOA".id, "PESSOA".nome, "PESSOA".sobrenome 
FROM "PESSOA"
2023-11-30 20:35:14,871 INFO sqlalchemy.engine.Engine [cached since 0.147s ago] ()
2 -> José -> Pocai
3 -> Cintia -> Carvalho
2023-11-30 20:35:14,872 INFO sqlalchemy.engine.Engine ROLLBACK


In [135]:
animal_cintia = Animal(nome='Pluto', pessoa=cintia)
animal_cintia

Animal(id=None, nome='Pluto', id_pessoa=None, pessoa='Cintia')

In [136]:
session = Session(engine)

session.add(animal_cintia)
session.commit()

2023-11-30 20:35:14,885 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-30 20:35:14,887 INFO sqlalchemy.engine.Engine INSERT INTO "ANIMAL" (nome, id_pessoa) VALUES (?, ?)
2023-11-30 20:35:14,888 INFO sqlalchemy.engine.Engine [generated in 0.00082s] ('Pluto', 3)
2023-11-30 20:35:14,889 INFO sqlalchemy.engine.Engine COMMIT


In [137]:
animal_cintia

2023-11-30 20:35:14,894 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2023-11-30 20:35:14,896 INFO sqlalchemy.engine.Engine SELECT "ANIMAL".id AS "ANIMAL_id", "ANIMAL".nome AS "ANIMAL_nome", "ANIMAL".id_pessoa AS "ANIMAL_id_pessoa" 
FROM "ANIMAL" 
WHERE "ANIMAL".id = ?
2023-11-30 20:35:14,896 INFO sqlalchemy.engine.Engine [generated in 0.00081s] (1,)
2023-11-30 20:35:14,898 INFO sqlalchemy.engine.Engine SELECT "PESSOA".id AS "PESSOA_id", "PESSOA".nome AS "PESSOA_nome", "PESSOA".sobrenome AS "PESSOA_sobrenome" 
FROM "PESSOA" 
WHERE "PESSOA".id = ?
2023-11-30 20:35:14,898 INFO sqlalchemy.engine.Engine [cached since 0.0793s ago] (3,)


Animal(id=1, nome='Pluto', id_pessoa=3, pessoa='Cintia')

In [138]:
cintia.animais[0]

2023-11-30 20:35:14,906 INFO sqlalchemy.engine.Engine SELECT "ANIMAL".id AS "ANIMAL_id", "ANIMAL".nome AS "ANIMAL_nome", "ANIMAL".id_pessoa AS "ANIMAL_id_pessoa" 
FROM "ANIMAL" 
WHERE ? = "ANIMAL".id_pessoa
2023-11-30 20:35:14,907 INFO sqlalchemy.engine.Engine [cached since 0.2185s ago] (3,)


Animal(id=1, nome='Pluto', id_pessoa=3, pessoa='Cintia')

In [139]:
session.close()

2023-11-30 20:35:14,914 INFO sqlalchemy.engine.Engine ROLLBACK
